##### 4.1

In [1]:
from statsmodels.stats.power import TTestIndPower
from math import ceil
power_analysis = TTestIndPower()
sample_size = ceil(power_analysis.solve_power(effect_size=0.5, power=0.8, alpha=0.05))
print(f'Sample Size: {sample_size}')

Sample Size: 64


##### 4.2

In [2]:
import numpy as np
from scipy.stats import ttest_ind

rng = np.random.default_rng(42)
sig = 0.05
reject_count = 0
experiment_count = 0

def generate_other_sample(sample):
    new_sample = rng.uniform(size=sample_size)
    new_sample = new_sample - new_sample.mean()  # Center the sample around its mean
    new_sample = sample.mean() + (sample.std() / new_sample.std()) * new_sample  # Scale to have the same std as sample1
    new_sample = new_sample + (sample.mean() - new_sample.mean())  # Adjust the mean to match sample1
    return new_sample

def generate_samples(sample_size):
    sample1 = rng.uniform(0, 1, sample_size)
    sample2 = generate_other_sample(sample1)
    return sample1, sample2

def perform_ttest(sample_size, sig):
    s1, s2 = generate_samples(sample_size)
    _, p = ttest_ind(s1, s2)
    return p < sig

def perform_experiment(sample_size, sig, test):
    global experiment_count
    experiment_count += 1
    if test(sample_size, sig):
        global reject_count
        reject_count += 1

for _ in range(1000):
    perform_experiment(sample_size, sig, perform_ttest)
print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')
print(f'Type 1 error rate: {reject_count / experiment_count}')

Experiment count: 1000
Reject count: 0
Type 1 error rate: 0.0


#### 4.3

In [3]:
effect_size = 0.5
experiment_count = 0
reject_count = 0

def perform_ttest_with_effect_size(sample_size, sig):
    s1 = rng.uniform(0,1,sample_size)
    s2 = rng.uniform(0,1,sample_size)
    s2 += effect_size
    _, p = ttest_ind(s1, s2)
    return p < sig

for _ in range(1000):
    perform_experiment(sample_size, sig, perform_ttest_with_effect_size)
print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')
print(f'Power: {reject_count / experiment_count}')

Experiment count: 1000
Reject count: 1000
Power: 1.0


#### 4.4 (2)

In [4]:
from multiprocessing.pool import ThreadPool as Pool
experiment_count = 0
reject_count = 0

def generate_cauchy_samples(sample_size):
    cauchy1 = rng.standard_cauchy(size=sample_size)
    cauchy2 = rng.standard_cauchy(size=sample_size)
    cauchy2 = cauchy2 - cauchy2.mean()  # Center the sample around its mean
    cauchy2 = cauchy1.mean() + (cauchy1.std() / cauchy2.std()) * cauchy2  # Scale to have the same std as sample1
    cauchy2 = cauchy2 + (cauchy1.mean() - cauchy2.mean())  # Adjust the mean to match sample1
    return cauchy1, cauchy2

def cauchy_ttest(sample_size, sig):
    c1, c2 = generate_cauchy_samples(sample_size)
    _, p = ttest_ind(c1, c2)
    return p < sig
pool = Pool()
for _ in range(1000):
    pool.apply_async(perform_experiment, args=(sample_size, sig, cauchy_ttest))
pool.close()
pool.join()

print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')
print(f'Type 1 error rate: {reject_count / experiment_count}')

Experiment count: 1000
Reject count: 0
Type 1 error rate: 0.0


#### 4.4 (2)

In [5]:
reject_count = 0
experiment_count = 0
effect_size = 0.5

def cauchy_ttest_effect_size(sample_size, sig):
    c1, c2 = rng.standard_cauchy(size=sample_size), rng.standard_cauchy(size=sample_size)
    c2 += effect_size
    _, p = ttest_ind(c1, c2)
    return p < sig

pool = Pool()
for _ in range(1000):
    pool.apply_async(perform_experiment, args=(sample_size, sig, cauchy_ttest_effect_size))
pool.close()
pool.join()

print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')
print(f'Power: {reject_count / experiment_count}')

Experiment count: 1000
Reject count: 29
Power: 0.029


4.4 (2) wilcoxon

In [6]:
from scipy.stats import mannwhitneyu

experiment_count = 0
reject_count = 0

def cauchy_mannwhitneyu(sample_size, sig):
    c1, c2 = rng.standard_cauchy(size=sample_size), rng.standard_cauchy(size=sample_size)
    c2 += effect_size
    _, p = mannwhitneyu(c1, c2)
    return p < sig
pool = Pool()
for _ in range(1000):
    pool.apply_async(perform_experiment, args=(sample_size, sig, cauchy_mannwhitneyu))
pool.close()
pool.join()

print(f'Experiment count: {experiment_count}')
print(f'Reject count: {reject_count}')
print(f'Power: {reject_count / experiment_count}')

Experiment count: 1000
Reject count: 317
Power: 0.317
